# 01 — Instance Generation

**Objectif :** générer des instances synthétiques réalistes du problème d’ordonnancement  
(tâches, machines, durées, deadlines, poids) et les sauvegarder pour les expériences.

Les instances seront stockées dans `data/instances/` au format JSON afin d’assurer
la **reproductibilité** des expériences Monte Carlo.


## 1) Pourquoi des instances synthétiques ?

Dans ce projet, nous utilisons des **instances synthétiques** afin de :
- contrôler la taille du problème (\(n\) tâches, \(m\) machines),
- ajuster la difficulté (durées, deadlines plus ou moins serrées),
- garantir la reproductibilité (grâce à une seed aléatoire),
- comparer équitablement différentes méthodes d’optimisation.

Ce choix est courant dans la littérature sur l’ordonnancement et l’optimisation Monte Carlo.


## 2) Paramètres d’une instance

Une instance est définie par :
- nombre de tâches : $n$
- nombre de machines : $m$
- matrice des durées : $p_{j,k}$
- deadlines : $d_j$ (optionnelles)
- poids / priorités : $w_j$ (optionnels)
- seed aléatoire (reproductibilité)

Dans la suite, on génère des instances avec :
- $p_{j,k}$ tirés aléatoirement dans un intervalle fixé,
- des deadlines proportionnelles à la charge moyenne du système.


In [1]:
import numpy as np
import json
import os
import pathlib
from datetime import datetime


## 3) Génération des durées de traitement

On considère des **machines parallèles non identiques**.

Les durées $p_{j,k}$ sont générées aléatoirement, par exemple selon une loi uniforme :

\[
p_{j,k} \sim \mathcal{U}(p_{\min}, p_{\max})
\]

Ce choix permet d’introduire une hétérogénéité réaliste entre machines.


In [2]:
def generate_processing_times(n_jobs, n_machines, p_min=1, p_max=20, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    return rng.integers(p_min, p_max + 1, size=(n_jobs, n_machines)).tolist()


## 4) Génération des deadlines

Les deadlines doivent être **ni trop laxistes ni trop strictes**.

Méthode utilisée :
1. estimer la charge moyenne du système,
2. fixer chaque deadline comme une fraction de cette charge.

Formellement, pour chaque tâche $j$ :
$$
d_j = \alpha \cdot \bar{C} + \varepsilon_j
$$

où :
- $\bar{C}$ est une estimation du makespan moyen,
- $\alpha$ contrôle la difficulté du problème,
- $\varepsilon_j$ est un bruit aléatoire modéré.


In [3]:
def generate_deadlines(processing_times, alpha=0.8, noise=0.2, rng=None):
    if rng is None:
        rng = np.random.default_rng()

    n_jobs = len(processing_times)
    n_machines = len(processing_times[0])

    avg_processing = np.mean(processing_times)
    estimated_cmax = (n_jobs * avg_processing) / n_machines

    deadlines = []
    for _ in range(n_jobs):
        eps = rng.uniform(-noise, noise) * estimated_cmax
        deadlines.append(max(0, alpha * estimated_cmax + eps))

    return deadlines


## 5) Poids et priorités (optionnels)

Pour modéliser des tâches plus ou moins importantes, on peut associer
un poids $w_j$ à chaque tâche.

Dans ce projet :
- les poids sont optionnels,
- ils peuvent être générés aléatoirement dans un intervalle discret.


In [4]:
def generate_weights(n_jobs, w_min=1, w_max=5, rng=None):
    if rng is None:
        rng = np.random.default_rng()
    return rng.integers(w_min, w_max + 1, size=n_jobs).tolist()


## 6) Construction complète d’une instance

On regroupe maintenant toutes les composantes pour construire une instance complète,
qui pourra être sauvegardée et réutilisée dans les notebooks suivants.


In [5]:
def generate_instance(
    name,
    n_jobs,
    n_machines,
    seed=0,
    with_deadlines=True,
    with_weights=False
):
    rng = np.random.default_rng(seed)

    processing_times = generate_processing_times(
        n_jobs, n_machines, rng=rng
    )

    deadlines = generate_deadlines(processing_times, rng=rng) if with_deadlines else None
    weights = generate_weights(n_jobs, rng=rng) if with_weights else None

    instance = {
        "name": name,
        "n_jobs": n_jobs,
        "n_machines": n_machines,
        "processing_times": processing_times,
        "deadlines": deadlines,
        "weights": weights,
        "meta": {
            "seed": seed,
            "created_at": datetime.now().isoformat()
        }
    }
    return instance


## 7) Sauvegarde de l’instance

Les instances sont sauvegardées au format JSON dans le dossier `data/instances/`.
Ce format est lisible, portable et compatible avec tous les notebooks suivants.


In [8]:
def save_instance(instance, directory="data/instances"):
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, f"{instance['name']}.json")
    with open(path, "w", encoding="utf-8") as f:
        json.dump(instance, f, indent=2)
    return path


In [9]:
instance = generate_instance(
    name="instance_50jobs_5machines",
    n_jobs=50,
    n_machines=5,
    seed=42,
    with_deadlines=True,
    with_weights=False
)

path = save_instance(instance)
print("Instance sauvegardée dans :", path)


Instance sauvegardée dans : data/instances/instance_50jobs_5machines.json


## 8) Conclusion

Nous avons généré une instance synthétique réaliste du problème d’ordonnancement
et l’avons sauvegardée pour les expériences futures.

➡️ Dans le prochain notebook (**02_Simulator_and_Metrics**), nous allons :
- charger une instance depuis `data/instances/`,
- simuler un planning donné,
- calculer les métriques (makespan, tardiness, score).
